<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/asyncio_threading_multiprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
n = lambda: random.randint(1, 100)

In [54]:
import random
import time

# Заполнение массива
start_time = time.time()
arr = [1 for _ in range(1000000)]
end_time = time.time()

# Вычисление суммы
sum_time = time.time()
sum_arr = sum(arr)
end_sum_time = time.time()

# Вывод времени выполнения
print(f"Время заполнения массива: {end_time - start_time:.6f} сек.")
print(f"Время вычисления суммы: {end_sum_time - sum_time:.6f} сек.")
print(f"Сумма элементов массива: {sum_arr}")


Время заполнения массива: 0.050765 сек.
Время вычисления суммы: 0.006530 сек.
Сумма элементов массива: 1000000


In [55]:
import threading
import random
import time

# Функция для заполнения части массива случайными числами
def fill_array_part(arr, start, end):
    for i in range(start, end):
        arr[i] = 1

# Функция для вычисления суммы части массива
def calculate_sum(arr, start, end, result):
    partial_sum = sum(arr[start:end])
    result.append(partial_sum)

# Создаем пустой массив из 1000000 элементов
arr = [0] * 1000000

# Разделяем массив на 10 частей
chunk_size = len(arr) // 10

# Создаем и запускаем потоки для заполнения массива
fill_threads = []
start_time_fill = time.time()
for i in range(10):
    thread = threading.Thread(target=fill_array_part, args=(arr, i*chunk_size, (i+1)*chunk_size))
    fill_threads.append(thread)
    thread.start()

# Ждем завершения всех потоков заполнения массива
for thread in fill_threads:
    thread.join()

end_time_fill = time.time()

# Создаем и запускаем потоки для вычисления суммы массива
sum_threads = []
results = []
start_time_sum = time.time()
for i in range(10):
    thread = threading.Thread(target=calculate_sum, args=(arr, i*chunk_size, (i+1)*chunk_size, results))
    sum_threads.append(thread)
    thread.start()

# Ждем завершения всех потоков вычисления суммы
for thread in sum_threads:
    thread.join()

end_time_sum = time.time()

total_sum = sum(results)

print(f"Время заполнения массива: {end_time_fill - start_time_fill} секунд")
print(f"Время вычисления суммы: {end_time_sum - start_time_sum} секунд")
print(f"Сумма элементов массива: {total_sum}")

Время заполнения массива: 0.062272071838378906 секунд
Время вычисления суммы: 0.02977156639099121 секунд
Сумма элементов массива: 1000000


In [56]:
import multiprocessing
import random
import time

# Функция для заполнения части массива случайными числами
def fill_array_part(arr, start, end):
    for i in range(start, end):
        arr[i] = 1

# Функция для вычисления суммы части массива
def calculate_sum(arr, start, end, result):
    partial_sum = sum(arr[start:end])
    result.put(partial_sum)

# Создаем пустой массив из 1000000 элементов
arr = multiprocessing.Array('i', [0]*1000000)

# Разделяем массив на 10 частей
chunk_size = len(arr) // 10

# Создаем и запускаем процессы для заполнения массива
fill_processes = []
start_time_fill = time.time()
for i in range(10):
    process = multiprocessing.Process(target=fill_array_part, args=(arr, i*chunk_size, (i+1)*chunk_size))
    fill_processes.append(process)
    process.start()

# Ждем завершения всех процессов заполнения массива
for process in fill_processes:
    process.join()

end_time_fill = time.time()

# Создаем и запускаем процессы для вычисления суммы массива
sum_processes = []
results = multiprocessing.Queue()
start_time_sum = time.time()
for i in range(10):
    process = multiprocessing.Process(target=calculate_sum, args=(arr, i*chunk_size, (i+1)*chunk_size, results))
    sum_processes.append(process)
    process.start()

# Ждем завершения всех процессов вычисления суммы
for process in sum_processes:
    process.join()

end_time_sum = time.time()

total_sum = 0
while not results.empty():
    total_sum += results.get()

print(f"Время заполнения: {end_time_fill - start_time_fill} секунд")
print(f"Время вычисления суммы: {end_time_sum - start_time_sum} секунд")
print(f"Сумма элементов массива: {total_sum}")

Время заполнения: 1.5937047004699707 секунд
Время вычисления суммы: 0.2033991813659668 секунд
Сумма элементов массива: 1000000


In [57]:
import asyncio
import random
import time

# Функция для заполнения части массива случайными числами
async def fill_array_part(arr, start, end):
    for i in range(start, end):
        arr[i] = 1

# Функция для вычисления суммы части массива
async def calculate_sum(arr, start, end):
    return sum(arr[start:end])

async def main():
    # Создаем пустой массив из 1000000 элементов
    arr = [0]*1000000

    # Разделяем массив на 10 частей
    chunk_size = len(arr) // 10

    # Запускаем асинхронные задачи для заполнения массива
    fill_tasks = []
    start_time_fill = time.time()
    for i in range(10):
        task = asyncio.create_task(fill_array_part(arr, i*chunk_size, (i+1)*chunk_size))
        fill_tasks.append(task)

    await asyncio.gather(*fill_tasks)
    end_time_fill = time.time()

    # Запускаем асинхронные задачи для вычисления суммы массива
    sum_tasks = []
    start_time_sum = time.time()
    for i in range(10):
        task = asyncio.create_task(calculate_sum(arr, i*chunk_size, (i+1)*chunk_size))
        sum_tasks.append(task)

    partial_sums = await asyncio.gather(*sum_tasks)
    end_time_sum = time.time()

    total_sum = sum(partial_sums)

    print(f"Время заполнения: {end_time_fill - start_time_fill} секунд")
    print(f"Время вычисления суммы: {end_time_sum - start_time_sum} секунд")
    print(f"Сумма элементов массива: {total_sum}")


await main()

Время заполнения: 0.059159278869628906 секунд
Время вычисления суммы: 0.011265039443969727 секунд
Сумма элементов массива: 1000000
